![banner](https://github.com/anopheles-genomic-surveillance/anopheles-genomic-surveillance.github.io/blob/master/docs/banner.jpg?raw=1)

***[Workshop 1](about) - Training course in data analysis for genomic surveillance of African malaria vectors***

---

# Module 4 - Analysing SNPs in the *Vgsc* gene

**Theme: Analysis**

In this module we're going to perform an analysis to discover single nucleotide polymorphisms (SNPs) in the voltage-gated sodium channel gene (*Vgsc*), which encodes the binding target for pyrethroid insecticides.

## Learning objectives

At the end of this module you will be able to:

* Discover mutations (SNPs) that could potentially cause pyrethroid target-site resistance.
* Compute SNP allele frequencies, i.e., how common are they in different mosquito cohorts?
* Perform analyses to compare SNP allele frequences between mosquitoes from different species, geographical locations and dates of collection.


## Lecture

### English

In [ ]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/i57txc-N0bM" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

### Français

In [ ]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/C9gVQTB2yLk" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

Please note that the code in the cells below might differ from that shown in the video. This can happen because Python packages and their dependencies change due to updates, necessitating tweaks to the code.

## Discovering SNPs in the *Vgsc* gene and computing SNP allele frequencies

First, let's set up the [`malariagen_data`](https://github.com/malariagen/malariagen-data-python) package to access MalariaGEN data in the cloud.

In [ ]:
%pip install -q --no-warn-conflicts malariagen_data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 29.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 548.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.4/24.4 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.5/302.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.

Note that authentication is required to access data through the package, please follow the instructions [here](https://malariagen.github.io/vector-data/vobs/vobs-data-access.html).

In [ ]:
import malariagen_data

In [ ]:
import pandas as pd

In [ ]:
ag3 = malariagen_data.Ag3(pre=True)
ag3

<MalariaGEN Ag3 API client>
Storage URL             : gs://vo_agam_release_master_us_central1/
Data releases available : 3.0, 3.1, 3.10, 3.11, 3.12, 3.13, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9
Results cache           : None
Cohorts analysis        : 20240717
AIM analysis            : 20220528
Site filters analysis   : dt_20200416
Software version        : malariagen_data 13.0.1
Client location         : South Carolina, United States (Google Cloud us-east1)
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact support@malariagen.net. For API documentation see 
https://malariagen.github.io/malariagen-data-python/v13.0.1/Ag3.html

To discover SNPs and compute allele frequencies, we're going to use the [`snp_allele_frequencies()` function](https://malariagen.github.io/malariagen-data-python/latest/generated/malariagen_data.ag3.Ag3.snp_allele_frequencies.html#malariagen_data.ag3.Ag3.snp_allele_frequencies). Let's have a look at the documentation for this function.

In [ ]:
ag3.snp_allele_frequencies?

To discover SNPs in the *Vgsc* gene, we need to define some parameters.

First, we need to decide which gene transcript to use when determining what SNP effects will be. Here we'll use the transcript with identifier "AGAP004707-RD".

In [ ]:
transcript = "AGAP004707-RD"

Next, to compute allele frequencies, we need to decide how our mosquitoes will be grouped into cohorts. There are different ways you can do this, for this analysis we'll group spatially by level 1 administrative divisions within countries, and temporally by year.

In [ ]:
cohorts = "admin1_year"

Next, we need to choose which samples to analyse. There are a number of different sample sets in the Ag3.0 data resource that we could use for this analysis. Let's check what's available.

In [ ]:
ag3.sample_sets()[['sample_set', 'study_id', 'sample_count']]

,sample_set,study_id,sample_count
0,AG1000G-AO,AG1000G-AO,81
1,AG1000G-BF-A,AG1000G-BF-1,181
2,AG1000G-BF-B,AG1000G-BF-1,102
3,AG1000G-BF-C,AG1000G-BF-2,13
4,AG1000G-CD,AG1000G-CD,76
...,...,...,...
80,1323-VO-GM-NGWA-VMF00235,1323-VO-GM-NGWA,188
81,1323-VO-GM-NGWA-VMF00242,1323-VO-GM-NGWA,1630
82,1329-VO-GA-CHRISTOPHE-VMF00228,1329-VO-GA-CHRISTOPHE,146
83,bergey-2019,bergey-2019,113


To keep things simple, for this module we'll focus on mosquitoes from Burkina Faso. There are three sample sets in the Ag3.0 resource providing data on mosquitoes from Burkina Faso.

In [ ]:
sample_sets = ['1191-VO-MULTI-OLOUGHLIN-VMF00106',
       '1191-VO-MULTI-OLOUGHLIN-VMF00140', '1296-VO-BF-DIABATE-VMF00272',
       '1314-VO-BF-KIENTEGA-KIMA-BF-2104', 'AG1000G-BF-A', 'AG1000G-BF-B',
       'AG1000G-BF-C', 'crawford-2016', 'fontaine-2015-rebuild',
       'tennessen-2021']

OK, now we're ready to run the analysis.

In [ ]:
snp_allele_freqs_df = ag3.snp_allele_frequencies(
    transcript=transcript,
    cohorts=cohorts,
    sample_sets=sample_sets,
    drop_invariant=False,
)
snp_allele_freqs_df

Load SNP genotypes:   0%|          | 0/244 [00:00<?, ?it/s]

Compute allele frequencies:   0%|          | 0/45 [00:00<?, ?it/s]

Compute SNP effects:   0%|          | 0/209001 [00:00<?, ?it/s]

pass_gamb_colu_arab  \
contig position ref_allele alt_allele aa_change                        
2L     2358158  A          C          M1L                       True   
                           T          M1L                       True   
                           G          M1V                       True   
       2358159  T          A          M1K                       True   
                           C          M1T                       True   
...                                                              ...   
       2431616  G          C          *2119S                    True   
                           T          *2119L                    True   
       2431617  A          C          *2119C                    True   
                           T          *2119C                    True   
                           G          *2119W                    True   

                                                 pass_gamb_colu  pass_arab  \
contig position ref_allele alt_allele aa_change                              
2L     2358158  A          C          M1L                  True       True   
                           T          M1L                  True       True   
                           G          M1V                  True       True   
       2358159  T          A          M1K                  True       True   
                           C          M1T                  True       True   
...                                                         ...        ...   
       2431616  G          C          *2119S               True       True   
                           T          *2119L               True       True   
       2431617  A          C          *2119C               True       True   
                           T          *2119C               True       True   
                           G          *2119W               True       True   

                                                 frq_BF-01_arab_2022  \
contig position ref_allele alt_allele aa_change                        
2L     2358158  A          C          M1L                        0.0   
                           T          M1L                        0.0   
                           G          M1V                        0.0   
       2358159  T          A          M1K                        0.0   
                           C          M1T                        0.0   
...                                                              ...   
       2431616  G          C          *2119S                     0.0   
                           T          *2119L                     0.0   
       2431617  A          C          *2119C                     0.0   
                           T          *2119C                     0.0   
                           G          *2119W                     0.0   

                                                 frq_BF-01_colu_2022  \
contig position ref_allele alt_allele aa_change                        
2L     2358158  A          C          M1L                        0.0   
                           T          M1L                        0.0   
                           G          M1V                        0.0   
       2358159  T          A          M1K                        0.0   
                           C          M1T                        0.0   
...                                                              ...   
       2431616  G          C          *2119S                     0.0   
                           T          *2119L                     0.0   
       2431617  A          C          *2119C                     0.0   
                           T          *2119C                     0.0   
                           G          *2119W                     0.0   

                                                 frq_BF-02_colu_2011  \
contig position ref_allele alt_allele aa_change                        
2L     2358158  A          C          M1L                        0.0   
       

In [ ]:
snp_gambiae_freqs_df = ag3.snp_allele_frequencies(
    transcript=transcript,
    cohorts=cohorts,
    sample_query="taxon == 'gambiae'",
    sample_sets=sample_sets,
    drop_invariant=False,
)
snp_gambiae_freqs_df

Load SNP genotypes:   0%|          | 0/204 [00:00<?, ?it/s]

Compute allele frequencies:   0%|          | 0/15 [00:00<?, ?it/s]

Compute SNP effects:   0%|          | 0/209001 [00:00<?, ?it/s]

pass_gamb_colu_arab  \
contig position ref_allele alt_allele aa_change                        
2L     2358158  A          C          M1L                       True   
                           T          M1L                       True   
                           G          M1V                       True   
       2358159  T          A          M1K                       True   
                           C          M1T                       True   
...                                                              ...   
       2431616  G          C          *2119S                    True   
                           T          *2119L                    True   
       2431617  A          C          *2119C                    True   
                           T          *2119C                    True   
                           G          *2119W                    True   

                                                 pass_gamb_colu  pass_arab  \
contig position ref_allele alt_allele aa_change                              
2L     2358158  A          C          M1L                  True       True   
                           T          M1L                  True       True   
                           G          M1V                  True       True   
       2358159  T          A          M1K                  True       True   
                           C          M1T                  True       True   
...                                                         ...        ...   
       2431616  G          C          *2119S               True       True   
                           T          *2119L               True       True   
       2431617  A          C          *2119C               True       True   
                           T          *2119C               True       True   
                           G          *2119W               True       True   

                                                 frq_BF-02_gamb_2022  \
contig position ref_allele alt_allele aa_change                        
2L     2358158  A          C          M1L                        0.0   
                           T          M1L                        0.0   
                           G          M1V                        0.0   
       2358159  T          A          M1K                        0.0   
                           C          M1T                        0.0   
...                                                              ...   
       2431616  G          C          *2119S                     0.0   
                           T          *2119L                     0.0   
       2431617  A          C          *2119C                     0.0   
                           T          *2119C                     0.0   
                           G          *2119W                     0.0   

                                                 frq_BF-07_gamb_2004  \
contig position ref_allele alt_allele aa_change                        
2L     2358158  A          C          M1L                        0.0   
                           T          M1L                        0.0   
                           G          M1V                        0.0   
       2358159  T          A          M1K                        0.0   
                           C          M1T                        0.0   
...                                                              ...   
       2431616  G          C          *2119S                     0.0   
                           T          *2119L                     0.0   
       2431617  A          C          *2119C                     0.0   
                           T          *2119C                     0.0   
                           G          *2119W                     0.0   

                                                 frq_BF-07_gamb_2022  \
contig position ref_allele alt_allele aa_change                        
2L     2358158  A          C          M1L                        0.0   
       

In [ ]:
snp_coluzzii_freqs_df = ag3.snp_allele_frequencies(
    transcript=transcript,
    cohorts=cohorts,
    sample_query="taxon == 'coluzzii'",
    sample_sets=sample_sets,
    drop_invariant=False,
)
snp_coluzzii_freqs_df

Load SNP genotypes:   0%|          | 0/232 [00:00<?, ?it/s]

Compute allele frequencies:   0%|          | 0/20 [00:00<?, ?it/s]

Compute SNP effects:   0%|          | 0/209001 [00:00<?, ?it/s]

pass_gamb_colu_arab  \
contig position ref_allele alt_allele aa_change                        
2L     2358158  A          C          M1L                       True   
                           T          M1L                       True   
                           G          M1V                       True   
       2358159  T          A          M1K                       True   
                           C          M1T                       True   
...                                                              ...   
       2431616  G          C          *2119S                    True   
                           T          *2119L                    True   
       2431617  A          C          *2119C                    True   
                           T          *2119C                    True   
                           G          *2119W                    True   

                                                 pass_gamb_colu  pass_arab  \
contig position ref_allele alt_allele aa_change                              
2L     2358158  A          C          M1L                  True       True   
                           T          M1L                  True       True   
                           G          M1V                  True       True   
       2358159  T          A          M1K                  True       True   
                           C          M1T                  True       True   
...                                                         ...        ...   
       2431616  G          C          *2119S               True       True   
                           T          *2119L               True       True   
       2431617  A          C          *2119C               True       True   
                           T          *2119C               True       True   
                           G          *2119W               True       True   

                                                 frq_BF-01_colu_2022  \
contig position ref_allele alt_allele aa_change                        
2L     2358158  A          C          M1L                        0.0   
                           T          M1L                        0.0   
                           G          M1V                        0.0   
       2358159  T          A          M1K                        0.0   
                           C          M1T                        0.0   
...                                                              ...   
       2431616  G          C          *2119S                     0.0   
                           T          *2119L                     0.0   
       2431617  A          C          *2119C                     0.0   
                           T          *2119C                     0.0   
                           G          *2119W                     0.0   

                                                 frq_BF-02_colu_2011  \
contig position ref_allele alt_allele aa_change                        
2L     2358158  A          C          M1L                        0.0   
                           T          M1L                        0.0   
                           G          M1V                        0.0   
       2358159  T          A          M1K                        0.0   
                           C          M1T                        0.0   
...                                                              ...   
       2431616  G          C          *2119S                     0.0   
                           T          *2119L                     0.0   
       2431617  A          C          *2119C                     0.0   
                           T          *2119C                     0.0   
                           G          *2119W                     0.0   

                                                 frq_BF-02_colu_2012  \
contig position ref_allele alt_allele aa_change                        
2L     2358158  A          C          M1L                   0.000000   
       

In [ ]:
snp_arabiensis_freqs_df = ag3.snp_allele_frequencies(
    transcript=transcript,
    cohorts=cohorts,
    sample_query="taxon == 'arabiensis'",
    sample_sets=sample_sets,
    drop_invariant=False,
)
snp_arabiensis_freqs_df

Load SNP genotypes:   0%|          | 0/124 [00:00<?, ?it/s]

Compute allele frequencies:   0%|          | 0/8 [00:00<?, ?it/s]

Compute SNP effects:   0%|          | 0/209001 [00:00<?, ?it/s]

pass_gamb_colu_arab  \
contig position ref_allele alt_allele aa_change                        
2L     2358158  A          C          M1L                       True   
                           T          M1L                       True   
                           G          M1V                       True   
       2358159  T          A          M1K                       True   
                           C          M1T                       True   
...                                                              ...   
       2431616  G          C          *2119S                    True   
                           T          *2119L                    True   
       2431617  A          C          *2119C                    True   
                           T          *2119C                    True   
                           G          *2119W                    True   

                                                 pass_gamb_colu  pass_arab  \
contig position ref_allele alt_allele aa_change                              
2L     2358158  A          C          M1L                  True       True   
                           T          M1L                  True       True   
                           G          M1V                  True       True   
       2358159  T          A          M1K                  True       True   
                           C          M1T                  True       True   
...                                                         ...        ...   
       2431616  G          C          *2119S               True       True   
                           T          *2119L               True       True   
       2431617  A          C          *2119C               True       True   
                           T          *2119C               True       True   
                           G          *2119W               True       True   

                                                 frq_BF-01_arab_2022  \
contig position ref_allele alt_allele aa_change                        
2L     2358158  A          C          M1L                        0.0   
                           T          M1L                        0.0   
                           G          M1V                        0.0   
       2358159  T          A          M1K                        0.0   
                           C          M1T                        0.0   
...                                                              ...   
       2431616  G          C          *2119S                     0.0   
                           T          *2119L                     0.0   
       2431617  A          C          *2119C                     0.0   
                           T          *2119C                     0.0   
                           G          *2119W                     0.0   

                                                 frq_BF-07_arab_2022  \
contig position ref_allele alt_allele aa_change                        
2L     2358158  A          C          M1L                        0.0   
                           T          M1L                        0.0   
                           G          M1V                        0.0   
       2358159  T          A          M1K                        0.0   
                           C          M1T                        0.0   
...                                                              ...   
       2431616  G          C          *2119S                     0.0   
                           T          *2119L                     0.0   
       2431617  A          C          *2119C                     0.0   
                           T          *2119C                     0.0   
                           G          *2119W                     0.0   

                                                 frq_BF-08_arab_2022  \
contig position ref_allele alt_allele aa_change                        
2L     2358158  A          C          M1L                        0.0   
       

The output from this function is a pandas DataFrame, where each row provides information about a SNP in the *Vgsc* gene.

Before we go further, to improve our understanding of what these data, let's look at some background.

## Grouping samples into cohorts

The MalariaGEN Ag3.0 data resource contain mosquito samples collected across large spatial and temporal scales, and from different mosquito species. When we want to run population genetic analyses on datasets like these, the data must be divided into biologically relevant **cohorts**, where a cohort is simply a group of samples we want to analyse together.

To help define cohorts and analyse these data, we have added some metadata for each sample about its time and place of collection and its species:

- **Spatially** - For most analyses we use administrative divisions to group the samples into cohorts. These give two levels of spatial resolution, where admin level 1 divides each country into a few large regions, while admin level 2 provides finer scale divisions such as provinces.

- **Temporally** - For each sample we provide the year, quarter and month of collection. Depending on your analysis, you can choose to group samples by year, by year and by year quarter, or by year and by month, although note that for some samples the collection month is missing.

- **Taxonomically** - Ag3.0 contains samples from different species in the *Anopheles gambiae* complex. To help with grouping by taxon, we have included a "taxon" field in the sample metadata.
   
Using these three dimensions, we have pre-defined six **cohort sets**, each of which groups samples into cohorts at different levels of spatio-temporal resolution. Within all cohort sets, samples are further subdivided by taxon.

- **admin1_year** - Cohorts obtained by grouping samples by admin level 1, collection year and taxon.
- **admin1_quarter** - Cohorts obtained by grouping samples by admin level 1, collection year and quarter, and taxon.
- **admin1_month** - Cohorts obtained by grouping samples by admin level 1, collection year and month, and taxon.
- **admin2_year** - Cohorts obtained by grouping samples by admin level 2, collection year and taxon.
- **admin2_quarter** - Cohorts obtained by grouping samples by admin level 2, collection year and quarter, and taxon.
- **admin2_month** - Cohorts obtained by grouping samples by admin level 2, collection year and month, and taxon.

Remember above we chose to use the "admin1_year" cohorts for our *Vgsc* analysis.

In [ ]:
cohorts

'admin1_year'

Let's now use this to understand the frequency columns in the SNP allele frequencies DataFrame. Here are the frequency column names:

In [ ]:
frequency_columns = [
    col for col in snp_allele_freqs_df.columns
    if col.startswith("frq_")
]
frequency_columns

['frq_BF-01_arab_2022',
 'frq_BF-01_colu_2022',
 'frq_BF-02_colu_2011',
 'frq_BF-02_colu_2012',
 'frq_BF-02_colu_2015',
 'frq_BF-02_colu_2016',
 'frq_BF-02_colu_2022',
 'frq_BF-02_gamb_2022',
 'frq_BF-02_gcx4_2011',
 'frq_BF-07_arab_2022',
 'frq_BF-07_colu_2022',
 'frq_BF-07_gamb_2004',
 'frq_BF-07_gamb_2022',
 'frq_BF-08_arab_2022',
 'frq_BF-08_colu_2022',
 'frq_BF-08_gamb_2022',
 'frq_BF-09_arab_2014',
 'frq_BF-09_arab_2015',
 'frq_BF-09_arab_2016',
 'frq_BF-09_arab_2019',
 'frq_BF-09_colu_2012',
 'frq_BF-09_colu_2014',
 'frq_BF-09_colu_2015',
 'frq_BF-09_colu_2016',
 'frq_BF-09_colu_2017',
 'frq_BF-09_colu_2018',
 'frq_BF-09_colu_2019',
 'frq_BF-09_colu_2021',
 'frq_BF-09_colu_2022',
 'frq_BF-09_gamb_2012',
 'frq_BF-09_gamb_2014',
 'frq_BF-09_gamb_2015',
 'frq_BF-09_gamb_2016',
 'frq_BF-09_gamb_2017',
 'frq_BF-09_gamb_2018',
 'frq_BF-09_gamb_2019',
 'frq_BF-09_gamb_2022',
 'frq_BF-12_arab_2022',
 'frq_BF-12_colu_2022',
 'frq_ML-2_colu_2014',
 'frq_ML-2_colu_2015',
 'frq_ML-2_gamb_20

Here "frq_" is used to mean that these columns contain frequencies.

The second part of the column name is either "BF-09" or "BF-07". These are [standard identifiers](https://en.wikipedia.org/wiki/ISO_3166-2) that refer to level 1 administrative divisions within countries, often called regions. "BF-07" is the Centre-Sud region within Burkina Faso, and "BF-09" is the Haut-Bassins region.

The third part refers to the species, and in this example is either "gamb" meaning *Anopheles gambiae* or "colu" meaning *Anopheles coluzzii*.

The final part is the year of collection, which in this example ie either 2004, 2012 or 2014.

Thus, e.g., "frq_BF-09_gamb_2012" means the frequency of the allele within the cohort of *Anopheles gambiae* mosquitoes from the BF-09 (Haut-Bassins) region collected in 2012.

## Single nucleotide polymorphisms (SNPs)

In the last module we learnt about reference genomes and gene annotations. Now one of the cool things we can do once we have an _Anopheles gambiae_ reference genome, is sequence the genomes of wild mosquitoes and compare them to the reference genome. By aligning an individual's sequencing reads to the reference, we can call its **genotype**.

The genotype is derived from how similar the individual is to the reference genome at each **nucleotide position**.

Where a nucleotide matches the reference genome, it will be called a **reference allele**, where "allele" is just a term for any kind of genetic variant. Where a nucleotide differs from the reference genome, it will be called an **alternative allele**.

Since, there are four possible nucleotides and one of these is the reference, there are always three possible alternative alleles for any given site. Another term for a genetic difference where two individuals differ at a single nucleotide position is **single nucleotide polymorphism** or **SNP**.

In [ ]:
%%html
<img width="60%" height="60%" src="https://vobs-resources.cog.sanger.ac.uk/training/img/workshop-1/w1m4-1.png"/>

This SNP information is displayed in the first four columns of the `snp_allele_frequencies()` index in the output dataframe. We can see each alternative allele represented by it's own row in our output. Let's take a look at just the index.

In [ ]:
snp_allele_freqs_df[[]]

Empty DataFrame
Columns: []
Index: [(2L, 2358158, A, C, M1L), (2L, 2358158, A, T, M1L), (2L, 2358158, A, G, M1V), (2L, 2358159, T, A, M1K), (2L, 2358159, T, C, M1T), (2L, 2358159, T, G, M1R), (2L, 2358160, G, A, M1I), (2L, 2358160, G, C, M1I), (2L, 2358160, G, T, M1I), (2L, 2358161, A, C, T2P), (2L, 2358161, A, T, T2S), (2L, 2358161, A, G, T2A), (2L, 2358162, C, A, T2N), (2L, 2358162, C, T, T2I), (2L, 2358162, C, G, T2S), (2L, 2358163, C, A, T2T), (2L, 2358163, C, T, T2T), (2L, 2358163, C, G, T2T), (2L, 2358164, G, A, E3K), (2L, 2358164, G, C, E3Q), (2L, 2358164, G, T, E3*), (2L, 2358165, A, C, E3A), (2L, 2358165, A, T, E3V), (2L, 2358165, A, G, E3G), (2L, 2358166, A, C, E3D), (2L, 2358166, A, T, E3D), (2L, 2358166, A, G, E3E), (2L, 2358167, G, A, D4N), (2L, 2358167, G, C, D4H), (2L, 2358167, G, T, D4Y), (2L, 2358168, A, C, D4A), (2L, 2358168, A, T, D4V), (2L, 2358168, A, G, D4G), (2L, 2358169, C, A, D4E), (2L, 2358169, C, T, D4D), (2L, 2358169, C, G, D4E), (2L, 2358170, T, A, S5T), (2L, 2358170, T, C, S5P), (2L, 2358170, T, G, S5A), (2L, 2358171, C, A, S5Y), (2L, 2358171, C, T, S5F), (2L, 2358171, C, G, S5C), (2L, 2358172, C, A, S5S), (2L, 2358172, C, T, S5S), (2L, 2358172, C, G, S5S), (2L, 2358173, G, A, D6N), (2L, 2358173, G, C, D6H), (2L, 2358173, G, T, D6Y), (2L, 2358174, A, C, D6A), (2L, 2358174, A, T, D6V), (2L, 2358174, A, G, D6G), (2L, 2358175, T, A, D6E), (2L, 2358175, T, C, D6D), (2L, 2358175, T, G, D6E), (2L, 2358176, T, A, S7T), (2L, 2358176, T, C, S7P), (2L, 2358176, T, G, S7A), (2L, 2358177, C, A, S7*), (2L, 2358177, C, T, S7L), (2L, 2358177, C, G, S7W), (2L, 2358178, G, A, S7S), (2L, 2358178, G, C, S7S), (2L, 2358178, G, T, S7S), (2L, 2358179, A, C, I8L), (2L, 2358179, A, T, I8L), (2L, 2358179, A, G, I8V), (2L, 2358180, T, A, I8K), (2L, 2358180, T, C, I8T), (2L, 2358180, T, G, I8R), (2L, 2358181, A, C, I8I), (2L, 2358181, A, T, I8I), (2L, 2358181, A, G, I8M), (2L, 2358182, T, A, S9T), (2L, 2358182, T, C, S9P), (2L, 2358182, T, G, S9A), (2L, 2358183, C, A, S9Y), (2L, 2358183, C, T, S9F), (2L, 2358183, C, G, S9C), (2L, 2358184, T, A, S9S), (2L, 2358184, T, C, S9S), (2L, 2358184, T, G, S9S), (2L, 2358185, G, A, E10K), (2L, 2358185, G, C, E10Q), (2L, 2358185, G, T, E10*), (2L, 2358186, A, C, E10A), (2L, 2358186, A, T, E10V), (2L, 2358186, A, G, E10G), (2L, 2358187, G, A, E10E), (2L, 2358187, G, C, E10D), (2L, 2358187, G, T, E10D), (2L, 2358188, G, A, E11K), (2L, 2358188, G, C, E11Q), (2L, 2358188, G, T, E11*), (2L, 2358189, A, C, E11A), (2L, 2358189, A, T, E11V), (2L, 2358189, A, G, E11G), (2L, 2358190, A, C, E11D), (2L, 2358190, A, T, E11D), (2L, 2358190, A, G, E11E), (2L, 2358191, G, A, E12K), ...]

[209001 rows x 0 columns]

## SNP effects - some SNPs may be more interesting than others

SNPs can have different effects depending on what the nucleotide change is and where in the genome it occurs.

In this analysis, we are interested in SNPs that affect protein structure, specifically those which will change the voltage-gated sodium-channel and could affect protein function and therefore insecticide resistance phenotype, so we need to look within the coding sequences (CDS) of the _Vgsc_ gene.

However, not all SNPs which fall in CDSs cause protein changes, because the genetic code has some redundancy, meaning that different nucleotide sequences can encode the same amino acid. If a SNP in a CDS does change the amino acid, it is called a **non-synonymous** (NS) or **missense** SNP, and if the SNP does not change the amino acid, it is called a **synonymous** SNP.  

Manually predicting SNP effects is quite involved, but `snp_allele_frequencies()` can predict them for us, let's look at how this is represented in our output DataFrame.


In [ ]:
snp_effects_df = snp_allele_freqs_df[["effect", "impact"]]
snp_effects_df

effect  \
contig position ref_allele alt_allele aa_change                          
2L     2358158  A          C          M1L                   START_LOST   
                           T          M1L                   START_LOST   
                           G          M1V                   START_LOST   
       2358159  T          A          M1K        NON_SYNONYMOUS_CODING   
                           C          M1T        NON_SYNONYMOUS_CODING   
...                                                                ...   
       2431616  G          C          *2119S                 STOP_LOST   
                           T          *2119L                 STOP_LOST   
       2431617  A          C          *2119C                 STOP_LOST   
                           T          *2119C                 STOP_LOST   
                           G          *2119W                 STOP_LOST   

                                                   impact  
contig position ref_allele alt_allele aa_change            
2L     2358158  A          C          M1L            HIGH  
                           T          M1L            HIGH  
                           G          M1V            HIGH  
       2358159  T          A          M1K        MODERATE  
                           C          M1T        MODERATE  
...                                                   ...  
       2431616  G          C          *2119S         HIGH  
                           T          *2119L         HIGH  
       2431617  A          C          *2119C         HIGH  
                           T          *2119C         HIGH  
                           G          *2119W         HIGH  

[209001 rows x 2 columns]

Let's look specifically at the genomic position where a SNP occurs which causes an insecticide-resistance mutation, also known as "kdr".

In [ ]:
snp_effects_df.loc[("2L", 2_422_652)]

effect    impact
ref_allele alt_allele aa_change                                 
A          C          L995F      NON_SYNONYMOUS_CODING  MODERATE
           T          L995F      NON_SYNONYMOUS_CODING  MODERATE
           G          L995L          SYNONYMOUS_CODING       LOW

For interest, let's count the number of SNPs we have by their effect.

In [ ]:
snp_allele_freqs_df.query("max_af > 0").groupby(["effect", "impact"]).size()

effect                 impact  
INTRONIC               MODIFIER    64105
NON_SYNONYMOUS_CODING  MODERATE     3094
SPLICE_CORE            HIGH          104
SPLICE_REGION          MODERATE      236
START_LOST             HIGH            1
STOP_GAINED            HIGH          226
STOP_LOST              HIGH            3
SYNONYMOUS_CODING      LOW          1067
dtype: int64

## SNP allele frequencies

Identifying the presence or absence of SNPs in wild caught mosquitoes is interesting, but the real value in generating SNP genotypes from large spatiotemporal collections of mosquitoes comes from the ability to see how groups of samples (cohorts) differ between geographical locations, species, and over time.  

One way to compare SNP differences between cohorts is to calculate and compare **SNP allele frequencies** at each position in the genome by dividing the number of times each SNP allele is found in the cohort by the total number of individuals present in the cohort (multiplied by 2 because each individual mosquito is diploid and so carries two genome copies).

In [ ]:
%%html
<img width="60%" height="60%" src="https://vobs-resources.cog.sanger.ac.uk/training/img/workshop-1/w1m4-2.png"/>

Let's take another look at the allele frequencies we computed above, focusing just on the frequency columns.

In [ ]:
snp_allele_freqs_df[frequency_columns + ['max_af']]

frq_BF-01_arab_2022  \
contig position ref_allele alt_allele aa_change                        
2L     2358158  A          C          M1L                        0.0   
                           T          M1L                        0.0   
                           G          M1V                        0.0   
       2358159  T          A          M1K                        0.0   
                           C          M1T                        0.0   
...                                                              ...   
       2431616  G          C          *2119S                     0.0   
                           T          *2119L                     0.0   
       2431617  A          C          *2119C                     0.0   
                           T          *2119C                     0.0   
                           G          *2119W                     0.0   

                                                 frq_BF-01_colu_2022  \
contig position ref_allele alt_allele aa_change                        
2L     2358158  A          C          M1L                        0.0   
                           T          M1L                        0.0   
                           G          M1V                        0.0   
       2358159  T          A          M1K                        0.0   
                           C          M1T                        0.0   
...                                                              ...   
       2431616  G          C          *2119S                     0.0   
                           T          *2119L                     0.0   
       2431617  A          C          *2119C                     0.0   
                           T          *2119C                     0.0   
                           G          *2119W                     0.0   

                                                 frq_BF-02_colu_2011  \
contig position ref_allele alt_allele aa_change                        
2L     2358158  A          C          M1L                        0.0   
                           T          M1L                        0.0   
                           G          M1V                        0.0   
       2358159  T          A          M1K                        0.0   
                           C          M1T                        0.0   
...                                                              ...   
       2431616  G          C          *2119S                     0.0   
                           T          *2119L                     0.0   
       2431617  A          C          *2119C                     0.0   
                           T          *2119C                     0.0   
                           G          *2119W                     0.0   

                                                 frq_BF-02_colu_2012  \
contig position ref_allele alt_allele aa_change                        
2L     2358158  A          C          M1L                   0.000000   
                           T          M1L                   0.000000   
                           G          M1V                   0.000000   
       2358159  T          A          M1K                   0.000000   
                           C          M1T                   0.000000   
...                                                              ...   
       2431616  G          C          *2119S                0.000000   
                           T          *2119L                0.007937   
       2431617  A          C          *2119C                0.000000   
                           T          *2119C                0.000000   
                           G          *2119W                0.000000   

                                                 frq_BF-02_colu_2015  \
contig position ref_allele alt_allele aa_change                        
2L     2358158  A          C          M1L                        0.0   
                           T          M1L                        0.0   
             

And let's inspect the frequencies for SNPs at a specific genomic position of interest.

In [ ]:
snp_allele_freqs_df.loc[("2L", 2_422_652), frequency_columns]

frq_BF-01_arab_2022  frq_BF-01_colu_2022  \
ref_allele alt_allele aa_change                                             
A          C          L995F                 0.000000             0.000000   
           T          L995F                 0.230769             0.390625   
           G          L995L                 0.000000             0.000000   

                                 frq_BF-02_colu_2011  frq_BF-02_colu_2012  \
ref_allele alt_allele aa_change                                             
A          C          L995F                 0.000000              0.00000   
           T          L995F                 0.722222              0.81746   
           G          L995L                 0.000000              0.00000   

                                 frq_BF-02_colu_2015  frq_BF-02_colu_2016  \
ref_allele alt_allele aa_change                                             
A          C          L995F                 0.000000             0.000000   
           T          L995F                 0.772727             0.764151   
           G          L995L                 0.000000             0.000000   

                                 frq_BF-02_colu_2022  frq_BF-02_gamb_2022  \
ref_allele alt_allele aa_change                                             
A          C          L995F                 0.000000                  0.0   
           T          L995F                 0.535714                  1.0   
           G          L995L                 0.000000                  0.0   

                                 frq_BF-02_gcx4_2011  frq_BF-07_arab_2022  \
ref_allele alt_allele aa_change                                             
A          C          L995F                 0.000000             0.000000   
           T          L995F                 0.487805             0.272727   
           G          L995L                 0.000000             0.000000   

                                 ...  frq_BF-09_gamb_2019  \
ref_allele alt_allele aa_change  ...                        
A          C          L995F      ...                  0.0   
           T          L995F      ...                  1.0   
           G          L995L      ...                  0.0   

                                 frq_BF-09_gamb_2022  frq_BF-12_arab_2022  \
ref_allele alt_allele aa_change                                             
A          C          L995F                      0.0                  0.0   
           T          L995F                      1.0                  0.1   
           G          L995L                      0.0                  0.0   

                                 frq_BF-12_colu_2022  frq_ML-2_colu_2014  \
ref_allele alt_allele aa_change                                            
A          C          L995F                 0.000000            0.000000   
           T          L995F                 0.368852            0.666667   
           G          L995L                 0.000000            0.000000   

                                 frq_ML-2_colu_2015  frq_ML-2_gamb_2004  \
ref_allele alt_allele aa_change                                           
A          C          L995F                0.000000                 0.0   
           T          L995F                0.745614                 0.0   
           G          L995L                0.000000                 0.0   

                                 frq_ML-2_gamb_2014  frq_ML-2_gamb_2015  \
ref_allele alt_allele aa_change                                           
A          C          L995F                     0.0                 0.0   
           T          L995F                     1.0                 1.0   
           G          L995L                     0.0                 0.0   

                                 frq_ZW18_quad_1986  
ref_allele alt_allele aa_change                      
A          C          L995F                     0.0  
           T          L995F                     0.0  
           G          L995L                     0.0  

[3 rows

## Visualising SNP allele frequencies via heatmaps

To make our SNP allele frequencies DataFrame easier to interpret, we can filter it down to just non-synonymous SNPs that are at frequency above 5% in at least one of our cohorts.

In [ ]:
ns_snps_df = snp_allele_freqs_df.query("effect == 'NON_SYNONYMOUS_CODING' and max_af >= 0.1")
ns_snps_df

pass_gamb_colu_arab  \
contig position ref_allele alt_allele aa_change                        
2L     2391228  G          C          V402L                     True   
                           T          V402L                     True   
       2416980  C          T          T791M                     True   
       2422651  T          C          L995S                     True   
       2422652  A          T          L995F                     True   
       2424739  C          A          T1166N                   False   
       2425417  A          T          N1345I                    True   
       2429616  A          G          I1527V                    True   
       2429617  T          C          I1527T                    True   
       2429623  T          G          F1529C                    True   
       2429694  G          A          A1553T                    True   
       2429745  A          T          N1570Y                    True   
       2429896  G          C          E1597Q                    True   
       2430424  G          T          A1746S                   False   
       2430794  A          T          N1845I                    True   
       2430817  G          A          V1853I                    True   
       2430863  T          C          I1868T                    True   
       2430880  C          T          P1874S                    True   
       2430881  C          T          P1874L                    True   
       2431061  C          T          A1934V                    True   
       2431079  T          C          I1940T                    True   
       2431499  C          T          P2080L                    True   

                                                 pass_gamb_colu  pass_arab  \
contig position ref_allele alt_allele aa_change                              
2L     2391228  G          C          V402L                True       True   
                           T          V402L                True       True   
       2416980  C          T          T791M                True       True   
       2422651  T          C          L995S                True       True   
       2422652  A          T          L995F                True       True   
       2424739  C          A          T1166N              False      False   
       2425417  A          T          N1345I               True       True   
       2429616  A          G          I1527V               True       True   
       2429617  T          C          I1527T               True       True   
       2429623  T          G          F1529C               True       True   
       2429694  G          A          A1553T               True       True   
       2429745  A          T          N1570Y               True       True   
       2429896  G          C          E1597Q               True       True   
       2430424  G          T          A1746S               True      False   
       2430794  A          T          N1845I               True       True   
       2430817  G          A          V1853I               True       True   
       2430863  T          C          I1868T               True       True   
       2430880  C          T          P1874S               True       True   
       2430881  C          T          P1874L               True       True   
       2431061  C          T          A1934V               True       True   
       2431079  T          C          I1940T               True       True   
       2431499  C          T          P2080L               True       True   

                                                 frq_BF-01_arab_2022  \
contig position ref_allele alt_allele aa_change                        
2L     2391228  G          C          V402L                 0.000000   
                           T          V402L                 0.000000   
       2416980  C          T          T791M                 0.000000   
       2422651  T          C          L995S                 0.076923   
       2422652 

In [ ]:
ns_snps_gambiae_df = snp_gambiae_freqs_df.query("effect == 'NON_SYNONYMOUS_CODING' and max_af >= 0.1")
ns_snps_gambiae_df

pass_gamb_colu_arab  \
contig position ref_allele alt_allele aa_change                        
2L     2416980  C          T          T791M                     True   
       2422652  A          T          L995F                     True   
       2429616  A          G          I1527V                    True   
       2429745  A          T          N1570Y                    True   
       2430424  G          T          A1746S                   False   
       2430817  G          A          V1853I                    True   
       2430863  T          C          I1868T                    True   
       2430881  C          T          P1874L                    True   

                                                 pass_gamb_colu  pass_arab  \
contig position ref_allele alt_allele aa_change                              
2L     2416980  C          T          T791M                True       True   
       2422652  A          T          L995F                True       True   
       2429616  A          G          I1527V               True       True   
       2429745  A          T          N1570Y               True       True   
       2430424  G          T          A1746S               True      False   
       2430817  G          A          V1853I               True       True   
       2430863  T          C          I1868T               True       True   
       2430881  C          T          P1874L               True       True   

                                                 frq_BF-02_gamb_2022  \
contig position ref_allele alt_allele aa_change                        
2L     2416980  C          T          T791M                 0.387097   
       2422652  A          T          L995F                 1.000000   
       2429616  A          G          I1527V                0.000000   
       2429745  A          T          N1570Y                0.112903   
       2430424  G          T          A1746S                0.387097   
       2430817  G          A          V1853I                0.032258   
       2430863  T          C          I1868T                0.145161   
       2430881  C          T          P1874L                0.209677   

                                                 frq_BF-07_gamb_2004  \
contig position ref_allele alt_allele aa_change                        
2L     2416980  C          T          T791M                 0.000000   
       2422652  A          T          L995F                 0.076923   
       2429616  A          G          I1527V                0.000000   
       2429745  A          T          N1570Y                0.038462   
       2430424  G          T          A1746S                0.000000   
       2430817  G          A          V1853I                0.000000   
       2430863  T          C          I1868T                0.000000   
       2430881  C          T          P1874L                0.000000   

                                                 frq_BF-07_gamb_2022  \
contig position ref_allele alt_allele aa_change                        
2L     2416980  C          T          T791M                 0.361111   
       2422652  A          T          L995F                 0.972222   
       2429616  A          G          I1527V                0.000000   
       2429745  A          T          N1570Y                0.055556   
       2430424  G          T          A1746S                0.361111   
       2430817  G          A          V1853I                0.138889   
       2430863  T          C          I1868T                0.111111   
       2430881  C          T          P1874L                0.277778   

                                                 frq_BF-08_gamb_2022  \
contig position ref_allele alt_allele aa_change                        
2L     2416980  C          T          T791M                   0.6125   
       2422652  A          T          L995F                   1.0000   
       2429616  A          G          I1527V                  0.0000   
       2429745  A       

In [ ]:
ns_snps_coluzzii_df = snp_coluzzii_freqs_df.query("effect == 'NON_SYNONYMOUS_CODING' and max_af >= 0.1")
ns_snps_coluzzii_df

pass_gamb_colu_arab  \
contig position ref_allele alt_allele aa_change                        
2L     2391228  G          C          V402L                     True   
                           T          V402L                     True   
       2422652  A          T          L995F                     True   
       2425417  A          T          N1345I                    True   
       2429617  T          C          I1527T                    True   
       2429623  T          G          F1529C                    True   
       2429745  A          T          N1570Y                    True   
       2429896  G          C          E1597Q                    True   
       2430880  C          T          P1874S                    True   
       2430881  C          T          P1874L                    True   
       2431061  C          T          A1934V                    True   
       2431079  T          C          I1940T                    True   

                                                 pass_gamb_colu  pass_arab  \
contig position ref_allele alt_allele aa_change                              
2L     2391228  G          C          V402L                True       True   
                           T          V402L                True       True   
       2422652  A          T          L995F                True       True   
       2425417  A          T          N1345I               True       True   
       2429617  T          C          I1527T               True       True   
       2429623  T          G          F1529C               True       True   
       2429745  A          T          N1570Y               True       True   
       2429896  G          C          E1597Q               True       True   
       2430880  C          T          P1874S               True       True   
       2430881  C          T          P1874L               True       True   
       2431061  C          T          A1934V               True       True   
       2431079  T          C          I1940T               True       True   

                                                 frq_BF-01_colu_2022  \
contig position ref_allele alt_allele aa_change                        
2L     2391228  G          C          V402L                 0.000000   
                           T          V402L                 0.609375   
       2422652  A          T          L995F                 0.390625   
       2425417  A          T          N1345I                0.015625   
       2429617  T          C          I1527T                0.609375   
       2429623  T          G          F1529C                0.015625   
       2429745  A          T          N1570Y                0.031250   
       2429896  G          C          E1597Q                0.078125   
       2430880  C          T          P1874S                0.156250   
       2430881  C          T          P1874L                0.015625   
       2431061  C          T          A1934V                0.046875   
       2431079  T          C          I1940T                0.000000   

                                                 frq_BF-02_colu_2011  \
contig position ref_allele alt_allele aa_change                        
2L     2391228  G          C          V402L                 0.083333   
                           T          V402L                 0.166667   
       2422652  A          T          L995F                 0.722222   
       2425417  A          T          N1345I                0.027778   
       2429617  T          C          I1527T                0.250000   
       2429623  T          G          F1529C                0.000000   
       2429745  A          T          N1570Y                0.222222   
       2429896  G          C          E1597Q                0.027778   
       2430880  C          T          P1874S                0.055556   
       2430881  C          T          P1874L                0.055556   
       2431061  C          T          A1934V                0.083333   
  

In [ ]:
ns_snps_arabiensis_df = snp_arabiensis_freqs_df.query("effect == 'NON_SYNONYMOUS_CODING' and max_af >= 0.1")
ns_snps_arabiensis_df

pass_gamb_colu_arab  \
contig position ref_allele alt_allele aa_change                        
2L     2422651  T          C          L995S                     True   
       2422652  A          T          L995F                     True   
       2429694  G          A          A1553T                    True   

                                                 pass_gamb_colu  pass_arab  \
contig position ref_allele alt_allele aa_change                              
2L     2422651  T          C          L995S                True       True   
       2422652  A          T          L995F                True       True   
       2429694  G          A          A1553T               True       True   

                                                 frq_BF-01_arab_2022  \
contig position ref_allele alt_allele aa_change                        
2L     2422651  T          C          L995S                 0.076923   
       2422652  A          T          L995F                 0.230769   
       2429694  G          A          A1553T                0.230769   

                                                 frq_BF-07_arab_2022  \
contig position ref_allele alt_allele aa_change                        
2L     2422651  T          C          L995S                 0.227273   
       2422652  A          T          L995F                 0.272727   
       2429694  G          A          A1553T                0.272727   

                                                 frq_BF-08_arab_2022  \
contig position ref_allele alt_allele aa_change                        
2L     2422651  T          C          L995S                 0.181818   
       2422652  A          T          L995F                 0.136364   
       2429694  G          A          A1553T                0.136364   

                                                 frq_BF-09_arab_2014  \
contig position ref_allele alt_allele aa_change                        
2L     2422651  T          C          L995S                 0.541667   
       2422652  A          T          L995F                 0.416667   
       2429694  G          A          A1553T                0.416667   

                                                 frq_BF-09_arab_2015  \
contig position ref_allele alt_allele aa_change                        
2L     2422651  T          C          L995S                 0.545455   
       2422652  A          T          L995F                 0.420455   
       2429694  G          A          A1553T                0.414773   

                                                 frq_BF-09_arab_2016  \
contig position ref_allele alt_allele aa_change                        
2L     2422651  T          C          L995S                 0.642857   
       2422652  A          T          L995F                 0.357143   
       2429694  G          A          A1553T                0.357143   

                                                 frq_BF-09_arab_2019  ...  \
contig position ref_allele alt_allele aa_change                       ...   
2L     2422651  T          C          L995S                 0.305556  ...   
       2422652  A          T          L995F                 0.652778  ...   
       2429694  G          A          A1553T                0.652778  ...   

                                                   max_af     transcript  \
contig position ref_allele alt_allele aa_change                            
2L     2422651  T          C          L995S      0.642857  AGAP004707-RD   
       2422652  A          T          L995F      0.652778  AGAP004707-RD   
       2429694  G          A          A1553T     0.652778  AGAP004707-RD   

                                                                effect  \
contig position ref_allele alt_allele aa_change                          
2L     2422651  T          C          L995S      NON_SYNONYMOUS_CODING   
       2422652  A          T          L995F      NON_SYNONYMOUS_CODING   
       2429694  G          A          A1553T     NON_SYNONYMOUS_CODI

To make things even clearer, we have included a heatmap plotting function to style our filtered DataFrame, called [`plot_frequencies_heatmap()`](https://malariagen.github.io/malariagen-data-python/latest/generated/malariagen_data.ag3.Ag3.plot_frequencies_heatmap.html#malariagen_data.ag3.Ag3.plot_frequencies_heatmap).

In [ ]:
ag3.plot_frequencies_heatmap(ns_snps_df,max_len=None)

In [ ]:
ag3.plot_frequencies_heatmap(ns_snps_gambiae_df,max_len=None)

In [ ]:
ag3.plot_frequencies_heatmap(ns_snps_coluzzii_df,max_len=None)

In [ ]:
ag3.plot_frequencies_heatmap(ns_snps_arabiensis_df,max_len=None)

## Amino acid substitution frequencies

You might have noticed that there are two rows with `V402L` in our previous heatmap plot. This is because in Burkina Faso, we find two different alternative alleles at the same genomic position, both causing the same amino acid substitution (valine to leucine).

If we are just interested in amino acid change frequencies, for example, when looking at potential insecticide resistance conferring mutations, we might want to combine the frequencies of the two alleles which cause V402L. In this case, we can use the `aa_allele_frequencies()` function in exactly the same way as we used `snp_allele_frequencies()`.

In [ ]:
aa_allele_freqs_df = ag3.aa_allele_frequencies(
    transcript=transcript,
    cohorts=cohorts,
    sample_sets=sample_sets
)
aa_allele_freqs_df

Load SNP genotypes:   0%|          | 0/244 [00:00<?, ?it/s]

Compute allele frequencies:   0%|          | 0/45 [00:00<?, ?it/s]

Compute SNP effects:   0%|          | 0/68836 [00:00<?, ?it/s]

,,,frq_BF-01_arab_2022,frq_BF-01_colu_2022,frq_BF-02_colu_2011,frq_BF-02_colu_2012,frq_BF-02_colu_2015,frq_BF-02_colu_2016,frq_BF-02_colu_2022,frq_BF-02_gamb_2022,frq_BF-02_gcx4_2011,frq_BF-07_arab_2022,...,transcript,aa_pos,ref_allele,ref_aa,alt_aa,effect,impact,alt_allele,max_af,label
aa_change,contig,position,,,,,,,,,,,,,,,,,,,,,
M1V,2L,2358158,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,AGAP004707-RD,1.0,A,M,V,START_LOST,HIGH,G,0.050000,"M1V (2L:2,358,158 A>G)"
M1R,2L,2358159,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,AGAP004707-RD,1.0,T,M,R,NON_SYNONYMOUS_CODING,MODERATE,G,0.050000,"M1R (2L:2,358,159 T>G)"
M1I,2L,2358160,0.0,0.0,0.0,0.007937,0.015152,0.0,0.0,0.0,0.000000,0.0,...,AGAP004707-RD,1.0,G,M,I,NON_SYNONYMOUS_CODING,MODERATE,T,0.015152,"M1I (2L:2,358,160 G>T)"
T2I,2L,2358162,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,AGAP004707-RD,2.0,C,T,I,NON_SYNONYMOUS_CODING,MODERATE,T,0.031250,"T2I (2L:2,358,162 C>T)"
T2N,2L,2358162,0.0,0.0,0.0,0.000000,0.015152,0.0,0.0,0.0,0.000000,0.0,...,AGAP004707-RD,2.0,C,T,N,NON_SYNONYMOUS_CODING,MODERATE,A,0.015152,"T2N (2L:2,358,162 C>A)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T2115K,2L,2431604,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,AGAP004707-RD,2115.0,C,T,K,NON_SYNONYMOUS_CODING,MODERATE,A,0.009615,"T2115K (2L:2,431,604 C>A)"
V2118A,2L,2431613,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,AGAP004707-RD,2118.0,T,V,A,NON_SYNONYMOUS_CODING,MODERATE,C,0.004854,"V2118A (2L:2,431,613 T>C)"
*2119R,2L,2431615,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,AGAP004707-RD,2119.0,T,*,R,STOP_LOST,HIGH,C,0.007692,"*2119R (2L:2,431,615 T>C)"


Let's filter it again to just amino acid changes greater than 5% in at least one cohort. We don't need to filter for non-synonymous mutations this time as this function has already done that for us.


In [ ]:
aa_filt_df = aa_allele_freqs_df.query("max_af >= 0.1")
aa_filt_df

,,,frq_BF-01_arab_2022,frq_BF-01_colu_2022,frq_BF-02_colu_2011,frq_BF-02_colu_2012,frq_BF-02_colu_2015,frq_BF-02_colu_2016,frq_BF-02_colu_2022,frq_BF-02_gamb_2022,frq_BF-02_gcx4_2011,frq_BF-07_arab_2022,...,transcript,aa_pos,ref_allele,ref_aa,alt_aa,effect,impact,alt_allele,max_af,label
aa_change,contig,position,,,,,,,,,,,,,,,,,,,,,
V402L,2L,2391228,0.000000,0.609375,0.250000,0.158730,0.227273,0.226415,0.464286,0.000000,0.000000,0.000000,...,AGAP004707-RD,402.0,G,V,L,NON_SYNONYMOUS_CODING,MODERATE,"{C,T}",0.631148,"V402L (2L:2,391,228 G>{C,T})"
T791M,2L,2416980,0.000000,0.000000,0.000000,0.007937,0.030303,0.028302,0.000000,0.387097,0.000000,0.000000,...,AGAP004707-RD,791.0,C,T,M,NON_SYNONYMOUS_CODING,MODERATE,T,0.612500,"T791M (2L:2,416,980 C>T)"
L995S,2L,2422651,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.227273,...,AGAP004707-RD,995.0,T,L,S,NON_SYNONYMOUS_CODING,MODERATE,C,0.642857,"L995S (2L:2,422,651 T>C)"
L995F,2L,2422652,0.230769,0.390625,0.722222,0.817460,0.772727,0.764151,0.535714,1.000000,0.487805,0.272727,...,AGAP004707-RD,995.0,A,L,F,NON_SYNONYMOUS_CODING,MODERATE,T,1.000000,"L995F (2L:2,422,652 A>T)"
T1166N,2L,2424739,0.000000,0.000000,0.000000,0.007937,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,AGAP004707-RD,1166.0,C,T,N,NON_SYNONYMOUS_CODING,MODERATE,A,0.100000,"T1166N (2L:2,424,739 C>A)"
N1345I,2L,2425417,0.000000,0.015625,0.027778,0.015873,0.015152,0.009434,0.071429,0.000000,0.000000,0.000000,...,AGAP004707-RD,1345.0,A,N,I,NON_SYNONYMOUS_CODING,MODERATE,T,0.150000,"N1345I (2L:2,425,417 A>T)"
I1527V,2L,2429616,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,AGAP004707-RD,1527.0,A,I,V,NON_SYNONYMOUS_CODING,MODERATE,G,0.100000,"I1527V (2L:2,429,616 A>G)"
I1527T,2L,2429617,0.000000,0.609375,0.250000,0.158730,0.227273,0.235849,0.464286,0.000000,0.000000,0.000000,...,AGAP004707-RD,1527.0,T,I,T,NON_SYNONYMOUS_CODING,MODERATE,C,0.631148,"I1527T (2L:2,429,617 T>C)"
F1529C,2L,2429623,0.000000,0.015625,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,...,AGAP004707-RD,1529.0,T,F,C,NON_SYNONYMOUS_CODING,MODERATE,G,0.129834,"F1529C (2L:2,429,623 T>G)"


Now we can visualise these frequencies the same way we did before.

In [ ]:
ag3.plot_frequencies_heatmap(aa_filt_df)

From this new heatmap it's clear that there are two SNPs causing a V402L substitution, and the combined frequency of these is identical to that of the I1527T substitution in the *An. coluzzii* cohorts. This is a strong indication that these two mutations are strongly associated and may be working together to confer a selective advantage via insecticide resistance.

## Using sample queries

In some cases you may want to analyse data across a number of different sample sets, but select only samples from a single country or species. In this case there is a useful `sample_query` parameter available with the functions `snp_allele_frequencies()` and `aa_allele_frequencies()`. The `sample_query` parameter accepts a pandas query string, which will be applied to the sample metadata to select samples to use in the analysis.

Let's see it in action by computing amino acid allele frequencies for all *An. arabiensis* samples in the Ag3.0 data resource.

In [ ]:
aa_arab_freqs_df = ag3.aa_allele_frequencies(
    transcript=transcript,
    cohorts=cohorts,
    sample_sets=sample_sets,
    sample_query="taxon == 'arabiensis'",
)
aa_arab_freqs_df = aa_arab_freqs_df.query("max_af > 0.05")
ag3.plot_frequencies_heatmap(aa_arab_freqs_df)

Load SNP genotypes:   0%|          | 0/124 [00:00<?, ?it/s]

Compute allele frequencies:   0%|          | 0/8 [00:00<?, ?it/s]

Compute SNP effects:   0%|          | 0/12480 [00:00<?, ?it/s]

## Plotting SNP frequency time series

For surveillance it can be very informative to observice changes in allele frequencies over time for a given sampling location and species. Alleles under selection due to insecticide resistance will increase in frequency over time, and plotting the frequency data as a time series can help to see this. The `malariagen_data` package includes a function [`plot_frequencies_time_series()`](https://malariagen.github.io/malariagen-data-python/latest/generated/malariagen_data.ag3.Ag3.plot_frequencies_time_series.html#malariagen_data.ag3.Ag3.plot_frequencies_time_series) for plotting frequency time series. These are a little more advanced to use, but we'll illustrate them below.

N.B., in the Ag3.0 data resource there is only one country where we have data from multiple time points, which is Burkina Faso. However, data in future releases will contain a number of time series, and so it's useful to illustrate this functionality now.

First we need to compute the SNP allele frequencies in our gene of interest. To obtain the necessary data for time series plotting, we use a slightly different function for computing the frequencies. Don't worry too much about this for the moment, we will revisit later in the course.

In [ ]:
ds_snps_bf = ag3.aa_allele_frequencies_advanced(
    transcript=transcript,
    area_by="admin1_iso",  # group samples in space by admin level 1
    period_by="year",  # group samples in time by year
    sample_sets=sample_sets,
    sample_query="country == 'Burkina Faso'",
    variant_query="max_af > 0.1",  # only keep variants above 5% frequency in at least one cohort
)
ds_snps_bf

Load SNP genotypes:   0%|          | 0/224 [00:00<?, ?it/s]

Compute SNP allele frequencies:   0%|          | 0/39 [00:00<?, ?it/s]

Compute SNP effects:   0%|          | 0/63658 [00:00<?, ?it/s]

<xarray.Dataset> Size: 34kB
Dimensions:                 (cohorts: 39, variants: 18)
Dimensions without coordinates: cohorts, variants
Data variables: (12/31)
    cohort_area             (cohorts) object 312B 'BF-01' 'BF-07' ... 'BF-02'
    cohort_label            (cohorts) object 312B 'BF-01_arab_2022' ... 'BF-0...
    cohort_lat_max          (cohorts) float64 312B 13.03 11.22 ... 11.24 10.7
    cohort_lat_mean         (cohorts) float64 312B 13.03 11.22 ... 11.24 10.7
    cohort_lat_min          (cohorts) float64 312B 13.03 11.22 ... 11.23 10.7
    cohort_lon_max          (cohorts) float64 312B -3.014 -1.02 ... -4.473 -4.8
    ...                      ...
    variant_label           (variants) object 144B 'V402L (2L:2,391,228 G>{C,...
    variant_max_af          (variants) float64 144B 0.6311 0.6125 ... 0.2683
    variant_position        (variants) int32 72B 2391228 2416980 ... 2431499
    variant_ref_aa          (variants) object 144B 'V' 'T' 'L' ... 'A' 'I' 'P'
    variant_ref_allele      (variants) object 144B 'G' 'C' 'T' ... 'C' 'T' 'C'
    variant_transcript      (variants) object 144B 'AGAP004707-RD' ... 'AGAP0...
Attributes:
    title:    AGAP004707-RD (Vgsc/para) SNP frequencies

Now let's plot the time series.

In [ ]:
ag3.plot_frequencies_time_series(ds_snps_bf)

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/usr/local/lib/python3.10/dist-packages/

Note that this plot is interactive, you can hover over points for more information, and zoom into regions of the plot.

In this plot we have data from two regions, BF-09 (Haut-Bassins) and BF-07 (Centre-Sud). In BF-09 we have data from 2012 and 2014, and so we have the beginning of a time series. In BF-07 we have data only from 2004, so just a single point in time.

There are 3 sub-plots, faceted by taxon and area to allow for comparison. The error bars in the plots show 95% confidence intervals for the frequency estimates. In BF-09 where we have data from 2012 and 2014, it looks like all SNP frequencies were similar between 2012 and 2014 in both *An. gambiae* and *An. coluzzii*, with some known insecticide resistance alleles such as L995F already at fixation or very high frequency.

It's interesting to note that in BF-07 region in 2004 there were no SNPs above 8% frequency, including the L995F SNP. This suggests that L995F rose in frequency in Burkina Faso between 2004 and 2012, although we cannot draw strong conclusions here because we have data from different regions.

## Plotting SNP frequency maps

Another analysis we can perform is to make geographical comparisons, by plotting SNP frequencies on a map. We can do this via functions in the `malariagen_data` package. Let's demonstrate, using all data in the Ag3.0 resource.

In [ ]:
ds_snps = ag3.aa_allele_frequencies_advanced(
    transcript=transcript,
    area_by="admin1_iso",  # group samples in space by admin level 1
    period_by="year",  # group samples in time by year
    sample_sets=sample_sets,
    variant_query="max_af > 0.05",  # only keep variants above 5% frequency in at least one cohort
)
ds_snps

Load SNP genotypes:   0%|          | 0/244 [00:00<?, ?it/s]

Compute SNP allele frequencies:   0%|          | 0/45 [00:00<?, ?it/s]

Compute SNP effects:   0%|          | 0/68836 [00:00<?, ?it/s]

<xarray.Dataset> Size: 90kB
Dimensions:                 (cohorts: 45, variants: 45)
Dimensions without coordinates: cohorts, variants
Data variables: (12/31)
    cohort_area             (cohorts) object 360B 'BF-01' 'BF-07' ... 'ZW18'
    cohort_label            (cohorts) object 360B 'BF-01_arab_2022' ... 'ZW18...
    cohort_lat_max          (cohorts) float64 360B 13.03 11.22 ... 10.7 -21.05
    cohort_lat_mean         (cohorts) float64 360B 13.03 11.22 ... 10.7 -21.05
    cohort_lat_min          (cohorts) float64 360B 13.03 11.22 ... 10.7 -21.05
    cohort_lon_max          (cohorts) float64 360B -3.014 -1.02 ... -4.8 31.67
    ...                      ...
    variant_label           (variants) object 360B 'E49G (2L:2,358,303 A>G)' ...
    variant_max_af          (variants) float64 360B 0.05556 0.05556 ... 0.2683
    variant_position        (variants) int32 180B 2358303 2385725 ... 2431499
    variant_ref_aa          (variants) object 360B 'E' 'L' 'R' ... 'A' 'I' 'P'
    variant_ref_allele      (variants) object 360B 'A' 'C' 'G' ... 'C' 'T' 'C'
    variant_transcript      (variants) object 360B 'AGAP004707-RD' ... 'AGAP0...
Attributes:
    title:    AGAP004707-RD (Vgsc/para) SNP frequencies

Take a look at the variants we've found.

In [ ]:
ds_snps["variant_label"].values

array(['E49G (2L:2,358,303 A>G)', 'L218F (2L:2,385,725 C>T)',
       'R254K (2L:2,390,177 G>A)', 'V256M (2L:2,390,182 G>A)',
       'E286* (2L:2,390,272 G>T)', 'T297I (2L:2,390,306 C>T)',
       'V402L (2L:2,391,228 G>{C,T})', 'V702I (2L:2,407,981 G>A)',
       'G706S (2L:2,407,993 G>A)', 'H721N (2L:2,408,115 C>A)',
       'H721Q (2L:2,408,117 C>A)', 'H727N (2L:2,408,133 C>A)',
       'T791M (2L:2,416,980 C>T)', 'Y821* (2L:2,417,071 C>A)',
       'T826I (2L:2,417,198 C>T)', 'G870R (2L:2,417,329 G>{A,C})',
       'G870V (2L:2,417,330 G>T)', 'L995S (2L:2,422,651 T>C)',
       'L995F (2L:2,422,652 A>T)', 'T1166N (2L:2,424,739 C>A)',
       'N1345I (2L:2,425,417 A>T)', 'Y1463F (2L:2,429,355 A>T)',
       'M1470I (2L:2,429,377 G>{A,T})', 'D1473Y (2L:2,429,384 G>T)',
       'Q1493H (2L:2,429,446 A>T)', 'I1527V (2L:2,429,616 A>G)',
       'I1527T (2L:2,429,617 T>C)', 'F1529C (2L:2,429,623 T>G)',
       'A1553T (2L:2,429,694 G>A)', 'N1570Y (2L:2,429,745 A>T)',
       'E1597Q (2L:2,429,896 G>C)

To explore these data a bit more interactively, there is also a function [`plot_frequencies_interactive_map()`](https://malariagen.github.io/malariagen-data-python/latest/generated/malariagen_data.ag3.Ag3.plot_frequencies_interactive_map.html#malariagen_data.ag3.Ag3.plot_frequencies_interactive_map) to create a map with some controls to select the taxon, period and variant of interest. Here's an example of how to use it.

In [ ]:
ag3.plot_frequencies_interactive_map(ds_snps)

NameError: name 'ag3' is not defined

N.B., you will need to run this notebook for yourself in colab in order for the interactive map to work.

## Well done!

In this module we have learnt how to analyse SNP mutations in the target of pyrethroid insecticides, the voltage-gated sodium-channel. We have calculated the allele frequencies of the SNPs in cohorts of mosquitoes and learnt how to filter and plot them for ease of interpretation.

## Practical exercises

### English

1. Open this notebook in Google Colab and run it for yourself from top to bottom. Hint: click the rocket icon at the top of the page, then select “Colab” from the drop-down menu.
2. Looking at the heatmap output (either amino acid or SNP), can you spot a relationship between the `V402L` and `L995F` frequencies? If so, what is it?
2. Re-run the whole analysis but using the Ghanaian sample set. Hint: Try `sample_sets = "AG1000G-GH"`. Or any other samples of interest.
3. What are the cohorts for this new sample set? Hint: see `frequency_columns`.
4. Above, we looked at the _kdr_ "West" SNP position, compare and contrast this with the _kdr_ "East" SNP position. What is the amino acid change for _kdr_ "East"? Hint: The position is 2422651.
5. For the Ghanaian sample set, add the a y_label which says "aa change" to the amino acid frequency heatmap. Hint: you can view all the function's parameters with `ag3.aa_allele_frequencies?`.
6. Remove the colorbar from the same heatmap. Hint: `False`.
7. Is the same relationship between `V402L` and `L995F` frequencies present in Ghana? What might be an evolutionary interpretation of this relationship?


### Français

1. [Ouvrir ce notebook dans Google Colab](https://colab.research.google.com/github/anopheles-genomic-surveillance/anopheles-genomic-surveillance.github.io/blob/master/docs/workshop-1/module-4-vgsc-snps.ipynb) et l'exécuter vous-même du début à la fin. Indice: cliquer sur l'icône fusée (<i class="fas fa-rocket"></i>) et sélectionner "Colab" dans le menu déroulant.
2. En regardant la heatmap (soit pour les SNPs soit pour les amino acides), pouvez vous détecter la relation entre les fréquences de `V402L` et `L995F`? Si oui, quelle est elle?
2. Re-exécuter toute l'analyse mais en utilisant les données ghanéennes. Indice: utiliser `sample_sets = "AG1000G-GH"`.
3. Quelles sont les cohortes de ce nouvel ensemble de données? Indice: regarder `frequency_columns`.
4. Plus haut, nous avons regardé la position SNP de _kdr_ "West", comparer et contraster la avec la position _kdr_ "East". Quelle est la différence en terme d'amino acides pour _kdr_ "East"? Indice: position = 2422651.
5. Pour l'ensemble de données ghanéennes, ajouter le y_label nommé "aa change" à la heatmap des fréquences des amino acides. Indice: tous les paramètres de la fonction peuvent être vus avec `ag3.aa_allele_frequencies?`.
6. Supprimer la barre de couleur de cette même heatmap. Indice: quelque chose doit être défini comme`False`.
7. Est-ce que la même relation entre les fréquences de `V402L` et `L995F` présente au Ghana? Quelle pourrait être l'interprétation en terme d'évolution de cette relation?
